In [43]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, TensorDataset
import time
from tqdm import tqdm

### Information
- We will do a few preliminary exercises and also build a character level MLP language model.
- This model will be similar to the model we did in class, except that we will have characters as tokens, not words.
- You will need a conda environment for this, here is general information on this.
 - https://docs.conda.io/projects/conda/en/latest/user-guide/install/index.html
 - PyTorch: https://anaconda.org/pytorch/pytorch
 
In the code below, FILL-IN the code necessary in the hint string provided.

### Preliminary exercises
- Please fill in the cells below with the asked for data.

In [4]:
torch.manual_seed(1)

In [5]:
# Create an embedding layer for a vocabulary of size 10 and the word vectors are each of dimension 5.
e = nn.Embedding(10, 5)

# Extract the embedding for the word whose token index is 3. What is the shape of this vector?
v = e(torch.tensor(3))

# Extract the weight matrix from the layer e.
# Create a linear layer (with no bias) of size 10 by 5 and set it's data to the embedding matrix.
l = nn.Linear(10, 5, bias=False)
l.weight = torch.nn.Parameter(e.weight.t())

# Insert inside of the assert below some sort of equality check.
# Hint: look up torch.all() and torch.eq()
assert(torch.all(torch.eq(l.weight.t(), e.weight)).item())

In [6]:
# Create a batch of size 2 with entries [0, 1, 2] and [2, 3, 4] in the data batch.
x = torch.stack([torch.tensor([0,1,2]), torch.tensor([2,3,4])])

In [7]:
assert(x.shape == (2, 3))

In [8]:
# What is the dimesion of this batch ran through the embeding layer?
assert(e(x).shape == (2, 3, 5))

### Constants and configs used below.

In [9]:
DEVICE = "cpu"
LR = 4.0
BATCH_SIZE = 16
NUM_EPOCHS = 5
MARKER = '.'
# N-gram level; P(w_t | w_{t-1}, ..., w_{t-n+1}).
n = 4
# Hidden layer dimension.
h = 20
# Word embedding dimension.
m = 20

### Get the dataset and the tokenizer.

In [10]:
class CharDataset(Dataset):
    def __init__(self, words, chars):
        self.words = words
        self.chars = chars
        # Inverse dictionaries mapping tokens to unique ids and the reverse.
        self.stoi = {ch:i for i,ch in enumerate(chars)}
        self.itos = {i:s for s,i in self.stoi.items()} # Inverse mapping.

    def __len__(self):
        return len(self.words)

    def contains(self, word):
        return word in self.words

    def get_vocab_size(self):
        return len(self.chars)

    def encode(self, word):
        return [self.stoi[c] for c in word]
    
    def decode(self, tokens):
        return ''.join(self.itos[i] for i in tokens)

    def __getitem__(self, idx):
        word = self.words[idx]
        return self.encode(word)

In [18]:
def create_datasets(window, input_file = 'names.txt'):
    """
    This takes a file of words and separates all the words.
    It then gets all the characters present in the universe of words and then ouputs the statistics. 
    """
    with open(input_file, 'r') as f:
        data = f.read()
    words = data.splitlines()
    words = [w.strip() for w in words] # This gets rid of any trailing and starting white spaces.
    words = [w for w in words if w] # Filter out all the empty words.
    
    chars = sorted(list(set(''.join(words)))) # This gets the universe of all characters.
    
    # Will force chars to have MARKER having index 0.
    chars= [MARKER] + chars
    
    words = [''.join((window-1)*[MARKER]) + word + ''.join((window-1)*[MARKER]) for word in words]
            
    print(f"The number of examples in the dataset: {len(words)}")
    print(f"The number of unique characters in the vocabulary: {len(chars)}")
    print(f"The vocabulary we have is: {''.join(chars)}")

    # Partition the input data into a training, validation, and the test set.
    out_of_sample_set_size = min(2000, int(len(words) * 0.1)) # We use 10% of the training set, or up to 2000 examples.
    test_set_size = 1500
    
    rp = torch.randperm(len(words)).tolist()
    
    train_words = [words[i] for i in rp[:-out_of_sample_set_size]]
    validation_words = [words[i] for i in rp[-out_of_sample_set_size:-test_set_size]]
    test_words = [words[i] for i in rp[-test_set_size:]]    
    
    print(f"We've split up the dataset into {len(train_words)}, {len(validation_words)}, {len(test_words)} training, validation, and test examples")

    # But the data in the data set objects.
    train_dataset = CharDataset(train_words, chars)
    validation_dataset = CharDataset(validation_words, chars)
    test_dataset = CharDataset(test_words, chars)

    return train_dataset, validation_dataset, test_dataset

In [19]:
train_dataset, validation_dataset, test_dataset = create_datasets(n)

The number of examples in the dataset: 32033
The number of unique characters in the vocabulary: 27
The vocabulary we have is: .abcdefghijklmnopqrstuvwxyz
We've split up the dataset into 30033, 500, 1500 training, validation, and test examples


## Explore the data

In [22]:
# Get the first word in "train_dataset"
train_dataset.words[0]

'...aasiya...'

In [51]:
train_dataset.stoi

{'.': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

### Get the dataloader

In [24]:
def create_dataloader(dataset, window):
    x_list = []
    y_list = []
    for i, word in enumerate(dataset):
        for j, _ in enumerate(word):
            if j + window > len(word) - 1:
                break
            word_window = word[j:j+window]
            x, y = word_window[:-1], word_window[-1]
            x_list.append(x)
            y_list.append(y)
            
    return DataLoader(
        TensorDataset(torch.tensor(x_list), torch.tensor(y_list)),
        BATCH_SIZE,
        shuffle=True
    )

In [25]:
train_dataloader = create_dataloader(train_dataset, n)
validation_dataloader = create_dataloader(validation_dataset, n)
test_dataloader = create_dataloader(test_dataset, n)

### Set up the model

In [33]:
# One of the first Neural language models!
class CharacterNeuralLanguageModel(nn.Module):
    def __init__(self, V, m, h, n):
        super(CharacterNeuralLanguageModel, self).__init__()
        
        # Vocabulary size.
        self.V = V
        
        # Embedding dimension, per word.
        self.m = m
        
        # Hidden dimension.
        self.h = h
        
        # N in "N-gram"
        self.n = n
        
        # Can you change all this stuff to use nn.Linear?
        # Ca also use nn.Parameter(torch.zeros(V, m)) for self.C but then we need one-hot and this is slow.
        self.C = nn.Embedding(V, m) 
        self.H = nn.Parameter(torch.zeros((n-1) * m, h))
        self.W = nn.Parameter(torch.zeros((n-1) * m, V))
        self.U = nn.Parameter(torch.zeros(h, V))
        
        self.b = torch.nn.Parameter(torch.ones(V))
        self.d = torch.nn.Parameter(torch.ones(h))
        
        self.init_weights()

    def init_weights(self):
        # Change this!
        initrange = 0.5
        
    def forward(self, x):
        
        # B X (n-1) X m 
        x = self.C(x)
        
        # B
        B = x.shape[0]
        
        # B X (n-1) * m
        x = x.view(B, -1)
    
        # B X V
        y = self.b + torch.matmul(x, self.W) + torch.matmul(nn.Tanh()(self.d + torch.matmul(x, self.H)), self.U)
        
        return y

### Set up the model.

In [34]:
criterion = torch.nn.CrossEntropyLoss().to(DEVICE)
model = CharacterNeuralLanguageModel(
    train_dataset.get_vocab_size(), m, h, n
).to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

### Train the model.

In [35]:
def calculate_perplexity(total_loss, total_batches):
    return torch.exp(torch.tensor(total_loss / total_batches)).item()

In [44]:
def train(dataloader, model, optimizer, criterion, epoch):
    model.train()
    total_loss, total_batches = 0.0, 0.0
    tl = []
    tb = []
    log_interval = 500

    for idx, (x, y) in tqdm(enumerate(dataloader)):
        optimizer.zero_grad()
        
        if not x.nelement():
            continue
        
        logits = model(x)
                        
        # Get the loss.
        loss = criterion(input=logits, target=y.squeeze(-1))

        # Do back propagation.
        loss.backward()
                        
        # Clip the gradients.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        
        # Do an optimization step.
        optimizer.step()
        total_loss += loss.item()
        total_batches += 1
        
        """
        The loss computation below is a bit off since the number of items per batch is not the same.
        We have that, for example, you might have 5 items in a batch and then 10 in another.
        Each original batch has 2 words.
        Then the answer from the first calculation will be S1/5 and also S2/10; you get 1/2(S1/5 + S2/10).
        What you reallly want is (S1 + S2) / 15.
        """
                
        if idx % log_interval == 0 and idx > 0:
            perplexity = calculate_perplexity(total_loss,  total_batches)
            print(
                "| epoch {:3d} "
                "| {:5d}/{:5d} batches "
                "| perplexity {:8.3f} "
                "| loss {:8.3f} "
                .format(
                    epoch,
                    idx,
                    len(dataloader),
                    perplexity,
                    total_loss / total_batches,
                )
            )
            total_loss, total_batches = 0.0, 0

In [45]:
def evaluate(dataloader, model, criterion):
    model.eval()
    total_loss, total_batches = 0.0, 0

    with torch.no_grad():
        for idx, (x, y) in enumerate(dataloader):
            logits = model(x)
            total_loss += criterion(input=logits, target=y.squeeze(-1)).item()
            total_batches += 1
    return total_loss / total_batches, calculate_perplexity(total_loss,  total_batches)

In [46]:
for epoch in range(1, NUM_EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader, model, optimizer, criterion, epoch)
    loss_val, perplexity_val = evaluate(validation_dataloader, model, criterion)
    scheduler.step()
    print("-" * 59)
    print(
        "| end of epoch {:3d} "
        "| time: {:5.2f}s "
        "| valid perplexity {:8.3f} "
        "| valid loss {:8.3f}".format(
            epoch,
            time.time() - epoch_start_time,
            perplexity_val,
            loss_val
        )
    )
    print("-" * 59)

print("Checking the results of test dataset.")
loss_test, perplexity_test = evaluate(test_dataloader, model, criterion)
print("test perplexity {:8.3f} | test loss {:8.3f} ".format(perplexity_test, loss_test))

1020it [00:00, 2664.65it/s]

| epoch   1 |   500/15255 batches | perplexity   10.395 | loss    2.341 
| epoch   1 |  1000/15255 batches | perplexity    9.218 | loss    2.221 


1863it [00:00, 2766.71it/s]

| epoch   1 |  1500/15255 batches | perplexity    8.665 | loss    2.159 
| epoch   1 |  2000/15255 batches | perplexity    8.565 | loss    2.148 


2995it [00:01, 2814.72it/s]

...

| epoch   5 | 14500/15255 batches | perplexity    7.210 | loss    1.975 


15035it [00:05, 2897.91it/s]

| epoch   5 | 15000/15255 batches | perplexity    7.519 | loss    2.017 


15255it [00:05, 2814.68it/s]


-----------------------------------------------------------
| end of epoch   5 | time:  5.45s | valid perplexity    7.565 | valid loss    2.024
-----------------------------------------------------------
Checking the results of test dataset.
test perplexity    7.464 | test loss    2.010 


## Generate some text.

In [59]:
def generate_word(model, dataset, window=n):
    generated_word = []
    context = [0] * (n-1)
    
    while True:
        logits = model(torch.tensor(context).view(1, -1))
                
        probs = nn.Softmax(dim=1)(logits)
        
        token_id = torch.multinomial(probs, num_samples=1).item()
        
        generated_word.append(token_id)
        
        context = context[1:] + [token_id]
        
        if token_id == 0:
            break
            
    return ''.join(dataset.decode(generated_word))

In [61]:
torch.manual_seed(1)
for _ in range(50):
    print(generate_word(model, train_dataset))

aha.
ele.
lia.
aldi.
jarorsse.
dez.
bemartti.
rielci.
revy.
madlais.
hoanda.
dacelia.
kalaliey.
chis.
mayas.
tya.
jon.
ama.
tze.
karies.
jos.
ahkl.
bamaka.
anyaamiush.
kazerher.
jami.
nnek.
maremellen.
toquyla.
nzygu.
enyl.
yanstram.
ahazoriexsunya.
sermontiroonn.
eifiah.
rosi.
rouivan.
ynn.
ahdityn.
jassavoli.
wun.
jayvarante.
nor.
ilyn.
marri.
allifare.
kalyi.
daslenshanna.
daniellaenimanaililah.
cyle.
